In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")

/Users/likhit/Documents/Coding/NLP/NLP_basics/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt = "What is Machine Learning is the best that humans have ever bulit  in there lives and tell us now check out of this really work can you  help me to start ?"
device = "mps" if torch.backends.mps.is_available() else "cpu"
inputs = tokenizer(prompt , return_tensors="pt").to(device)
model.to(device)
print(inputs["input_ids"].shape)
print(inputs["attention_mask"].shape)


torch.Size([1, 36])
torch.Size([1, 36])


In [3]:
# def fun(i):
#     print(f"The layer", i )
    
# for i , layers in enumerate(model.model.layers):
#     layers.self_attn.register_forward_hook(fun(i))

In [4]:
kv_cache = {}

def save_kv_from_attention(layer_idx):
    def hook(module, input, output):
        if isinstance(output, tuple) and len(output) > 1:
            k, v = output[1]
            kv_cache[f"layer_{layer_idx}"] = (k.detach().cpu(), v.detach().cpu())
            print(f"[Saved] layer {layer_idx}: K={k.shape}, V={v.shape}")
    return hook

for i, layer in enumerate(model.model.layers):
    layer.self_attn.register_forward_hook(save_kv_from_attention(i))

_ = model(**inputs, use_cache=True)

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
print("Total layers hooked:", len(model.model.layers))

fired = []
def record_fired(layer_idx):
    def hook(module, input, output):
        fired.append(layer_idx)
        print(f"[Hook fired] layer {layer_idx}")
    return hook

for i, layer in enumerate(model.model.layers):
    layer.self_attn.register_forward_hook(record_fired(i))

try:
    _ = model(**inputs, use_cache=True)
except Exception as e:
    print("Error:", e)

print("Layers that actually ran:", fired)

Total layers hooked: 18
[Layer 0] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 0
[Layer 1] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 1
[Layer 2] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 2
[Layer 3] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 3
[Layer 4] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 4
[Layer 5] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 5
[Layer 6] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 6
[Layer 7] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 7
[Layer 8] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 8
[Layer 9] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 9
[Layer 10] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fired] layer 10
[Layer 11] type(output) = <class 'tuple'>
   tuple length = 2
[Hook fire